## BRIA Background Removal v1.4 Model
#### https://huggingface.co/briaai/RMBG-1.4

In [20]:
pip install -qr https://huggingface.co/briaai/RMBG-1.4/resolve/main/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [22]:
import os
import json
from transformers import pipeline
from tqdm import tqdm
import time

In [23]:
def get_image_mask(image_path):
    pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True)
    pillow_mask = pipe(image_path, return_mask = True) # outputs a pillow mask
    # pillow_image = pipe(image_path) # applies mask on input and returns a pillow image
    return pillow_mask

In [24]:
def save_image(objectId, path, pillow_mask):
    filename_mask = path + str(objectId) + "mask.png"

    if not os.path.exists(filename_mask):
        pillow_mask.save(filename_mask)
    else:
        print(f"Skipped: {filename_mask} already exists.")

In [25]:
def save_BRIA_image(obj, path):
    image_path = obj["imageUrl"]
    objectId = obj["objectId"]

    filename_mask = path + str(objectId) + "mask.png"
    if os.path.exists(filename_mask):
            print(f"Skipped: {filename_mask} already exists.")
            return

    pillow_mask = get_image_mask(image_path)
    pillow_mask.save(filename_mask)
    # save_image(objectId, path, pillow_mask)

In [26]:
def run_batched_BRIA_processing(data, save_path, batch_size=10, delay=0):
    """
    Iterates through an array of objects and calls save_BRIA_image in batches.

    Args:
        data (list): List of objects with keys "objectId" and "imageUrl"
        save_path (str): Where to save output images
        batch_size (int): How many to process per batch
        delay (float): Optional delay between batches (in seconds)
    """
    total = len(data)

    for i in tqdm(range(0, total, batch_size), desc="Processing batches"):
        batch = data[i:i + batch_size]

        for obj in batch:
            try:
                save_BRIA_image(obj, save_path)
            except Exception as e:
                print(f"Error processing object {obj.get('objectId')}: {e}")

        if delay:
            time.sleep(delay)

#### TEST

In [27]:
with open("../objects_db_test.json", "r") as f:
    test_data = json.load(f)

In [28]:
# print(len(test_data['objects']))
test_objs = test_data['objects']

In [29]:
test_output_path = "../public/data/initial_test/BRIA/"

In [30]:
save_BRIA_image(test_objs[0], test_output_path)

Skipped: ../public/data/initial_test/BRIA/14318mask.png already exists.


In [31]:
run_batched_BRIA_processing(test_objs, test_output_path)

Processing batches: 100%|██████████| 28/28 [00:00<00:00, 9563.56it/s]

Skipped: ../public/data/initial_test/BRIA/14318mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/9480mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/16589mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/17732mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/551250mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/21355mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/820892mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/42229mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/9317mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/19067mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/208181mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/20804mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/9361mask.png already exists.
Skipped: ../public/data/initial_test/BRIA/6779mask.png already e

### Save images to folder for FE

In [32]:
with open("../objects_db.json", "r") as f:
    data = json.load(f)

objs = data['objects']

output_path = "../public/data/images/"

In [33]:
save_BRIA_image(objs[0], output_path)

Skipped: ../public/data/images/14318mask.png already exists.


In [34]:
save_BRIA_image(objs[0], output_path)

Skipped: ../public/data/images/14318mask.png already exists.


In [36]:
run_batched_BRIA_processing(objs, output_path)

Processing batches:   0%|          | 0/127 [00:00<?, ?it/s]

Skipped: ../public/data/images/14318mask.png already exists.
Skipped: ../public/data/images/9480mask.png already exists.
Skipped: ../public/data/images/16589mask.png already exists.
Skipped: ../public/data/images/17732mask.png already exists.
Skipped: ../public/data/images/551250mask.png already exists.
Skipped: ../public/data/images/21355mask.png already exists.
Skipped: ../public/data/images/820892mask.png already exists.
Skipped: ../public/data/images/42229mask.png already exists.
Skipped: ../public/data/images/9317mask.png already exists.
Skipped: ../public/data/images/19067mask.png already exists.
Skipped: ../public/data/images/208181mask.png already exists.
Skipped: ../public/data/images/20804mask.png already exists.
Skipped: ../public/data/images/9361mask.png already exists.
Skipped: ../public/data/images/6779mask.png already exists.
Skipped: ../public/data/images/6778mask.png already exists.
Skipped: ../public/data/images/1029mask.png already exists.
Skipped: ../public/data/ima

Device set to use mps:0
Device set to use mps:0


Error processing object 238928: cannot identify image file <_io.BytesIO object at 0x416b7d7b0>


Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Processing batches:  59%|█████▉    | 75/127 [00:09<00:06,  7.51it/s]Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Processing batches:  60%|█████▉    | 76/127 [00:32<00:28,  1.82it/s]Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Processing batches:  61%|██████    | 77/127 [00:54<00:55,  1.11s/it]

Skipped: ../public/data/images/9218mask.png already exists.
Skipped: ../public/data/images/9407mask.png already exists.
Skipped: ../public/data/images/9406mask.png already exists.
Skipped: ../public/data/images/807mask.png already exists.
Skipped: ../public/data/images/9411mask.png already exists.
Skipped: ../public/data/images/483856mask.png already exists.
Skipped: ../public/data/images/16001mask.png already exists.
Skipped: ../public/data/images/15595mask.png already exists.
Skipped: ../public/data/images/9324mask.png already exists.
Skipped: ../public/data/images/20737mask.png already exists.
Skipped: ../public/data/images/9323mask.png already exists.
Skipped: ../public/data/images/9367mask.png already exists.
Skipped: ../public/data/images/487944mask.png already exists.
Skipped: ../public/data/images/9479mask.png already exists.
Skipped: ../public/data/images/9394mask.png already exists.
Skipped: ../public/data/images/9395mask.png already exists.
Skipped: ../public/data/images/941

Device set to use mps:0
Processing batches: 100%|██████████| 127/127 [00:56<00:00,  2.25it/s]

Error processing object 238928: cannot identify image file <_io.BytesIO object at 0x120de7dd0>
Skipped: ../public/data/images/250258mask.png already exists.
Skipped: ../public/data/images/243971mask.png already exists.
Skipped: ../public/data/images/52937mask.png already exists.
Skipped: ../public/data/images/52938mask.png already exists.
Skipped: ../public/data/images/550071mask.png already exists.
Skipped: ../public/data/images/549375mask.png already exists.
Skipped: ../public/data/images/248849mask.png already exists.
Skipped: ../public/data/images/204619mask.png already exists.
Skipped: ../public/data/images/204620mask.png already exists.
Skipped: ../public/data/images/770924mask.png already exists.
Skipped: ../public/data/images/44247mask.png already exists.
Skipped: ../public/data/images/62470mask.png already exists.
Skipped: ../public/data/images/468422mask.png already exists.
Skipped: ../public/data/images/675791mask.png already exists.
Skipped: ../public/data/images/449686mask